## TCS iON RIO-125: Forecasting System - Project Demand of Products at a Retail Outlet Based on Historical Data

##1. Understanding the Data

######Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,KFold)
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
#import plotly.plotly as py
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
#!pip install xgboost
import xgboost as xgb
import copy
%matplotlib inline

######Reading the dataset into the Python Environment

In [2]:
#Loading data :
train = pd.read_csv('train.csv', parse_dates=['date'])
test = pd.read_csv('test.csv', parse_dates=['date'])
sample = pd.read_csv('sample_submission.csv')
print('Train shape:{}, Test shape:{}'.format(train.shape, test.shape))
train.head()

Train shape:(913000, 4), Test shape:(45000, 4)


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [3]:
sample.head()

,id,sales
0,0,52
1,1,52
2,2,52
3,3,52
4,4,52


In [4]:
print(test.shape)
test.head()

(45000, 4)


,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [5]:
print(train.shape)
train.head()

(913000, 4)


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [6]:
train['date']=pd.to_datetime(train['date'], infer_datetime_format=True)
test['date']=pd.to_datetime(test['date'], infer_datetime_format=True)
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['week'] = train['date'].dt.week
train['weekofyear'] = train['date'].dt.weekofyear
train['dayofweek'] = train['date'].dt.dayofweek
train['weekday'] = train['date'].dt.weekday
train['dayofyear'] = train['date'].dt.dayofyear
train['quarter'] = train['date'].dt.quarter
train['is_month_start'] = train['date'].dt.is_month_start
train['is_month_end'] =train['date'].dt.is_month_end
train['is_quarter_start'] = train['date'].dt.is_quarter_start
train['is_quarter_end'] = train['date'].dt.is_quarter_end
train['is_year_start'] = train['date'].dt.is_year_start
train['is_year_end'] = train['date'].dt.is_year_end
train['daily_avg']=train.groupby(['item','store','dayofweek'])['sales'].transform('mean')
train['monthly_avg']=train.groupby(['item','store','month'])['sales'].transform('mean')
train["mean_store_item_month"] = train.groupby(['month',"item","store"])["sales"].transform("mean")
train["item_month_sum"] = train.groupby(['month',"item"])["sales"].transform("sum") # total sales of that item  for all stores
train["store_month_sum"] = train.groupby(['month',"store"])["sales"].transform("sum") # total sales of that store  for all items
#train['store_item_shifted_365'] = train.groupby(["item","store"])['sales'].transform(lambda x:x.shift(365)) # sales for that 1 year  ago
#train["item-week_shifted_90"] = train.groupby(['weekofyear',"item"])["sales"].transform(lambda x:x.shift(12).sum()) # shifted total sales for that item 12 weeks (3 months) ago
#train['store_item_shifted_365'].fillna(train['store_item_shifted_365'].mode()[0], inplace=True)
daily_avg=train.groupby(['item','store','dayofweek'])['sales'].mean().reset_index()
monthly_avg=train.groupby(['item','store','month'])['sales'].mean().reset_index()
mean_store_item_month = train.groupby(['month','item','store'])['sales'].mean().reset_index()
item_month_sum=train.groupby(['month','item'])['sales'].sum().reset_index()
store_month_sum=train.groupby(['month','store'])['sales'].sum().reset_index()
#store_item_shifted_365=train.groupby(['item','store'])['sales'].shift(365).reset_index
#item_week_shifted_90=train.groupby(['weekofyear','item','dayofweek'])['sales'].shift(12).reset_index()
train.dtypes

<ipython-input-6-e795590c4741>:6: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.

<ipython-input-6-e795590c4741>:7: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



date                     datetime64[ns]
store                             int64
item                              int64
sales                             int64
year                              int64
month                             int64
day                               int64
week                              int64
weekofyear                        int64
dayofweek                         int64
weekday                           int64
dayofyear                         int64
quarter                           int64
is_month_start                     bool
is_month_end                       bool
is_quarter_start                   bool
is_quarter_end                     bool
is_year_start                      bool
is_year_end                        bool
daily_avg                       float64
monthly_avg                     float64
mean_store_item_month           float64
item_month_sum                    int64
store_month_sum                   int64
dtype: object

In [7]:
#Create mask for boolean values for train dataset
train['is_month_start'] = train['is_month_start'].replace({True: 1, False: 0})
train['is_month_end'] = train['is_month_end'].replace({True: 1, False: 0})
train['is_quarter_start'] = train['is_quarter_start'].replace({True: 1, False: 0})
train['is_quarter_end'] = train['is_quarter_end'].replace({True: 1, False: 0})
train['is_year_start'] = train['is_year_start'].replace({True: 1, False: 0})
train['is_year_end'] = train['is_year_end'].replace({True: 1, False: 0})

#convert float to int
train['daily_avg']=train['daily_avg'].astype(np.int64)
train['monthly_avg']=train['monthly_avg'].astype(np.int64)
train['mean_store_item_month']=train['mean_store_item_month'].astype(np.int64)
#train['store_item_shifted_365']=train['store_item_shifted_365'].astype(np.int64)
train.dtypes

date                     datetime64[ns]
store                             int64
item                              int64
sales                             int64
year                              int64
month                             int64
day                               int64
week                              int64
weekofyear                        int64
dayofweek                         int64
weekday                           int64
dayofyear                         int64
quarter                           int64
is_month_start                    int64
is_month_end                      int64
is_quarter_start                  int64
is_quarter_end                    int64
is_year_start                     int64
is_year_end                       int64
daily_avg                         int64
monthly_avg                       int64
mean_store_item_month             int64
item_month_sum                    int64
store_month_sum                   int64
dtype: object

In [8]:
train.head()

,date,store,item,sales,year,month,day,week,weekofyear,dayofweek,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,daily_avg,monthly_avg,mean_store_item_month,item_month_sum,store_month_sum
0,2013-01-01,1,1,13,2013,1,1,1,1,1,...,0,1,0,1,0,18,13,13,22987,249352
1,2013-01-02,1,1,11,2013,1,2,1,1,2,...,0,0,0,0,0,18,13,13,22987,249352
2,2013-01-03,1,1,14,2013,1,3,1,1,3,...,0,0,0,0,0,19,13,13,22987,249352
3,2013-01-04,1,1,13,2013,1,4,1,1,4,...,0,0,0,0,0,21,13,13,22987,249352
4,2013-01-05,1,1,10,2013,1,5,1,1,5,...,0,0,0,0,0,22,13,13,22987,249352


In [9]:
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['week'] = test['date'].dt.week
test['weekofyear'] = test['date'].dt.weekofyear
test['dayofweek'] = test['date'].dt.dayofweek
test['weekday'] = test['date'].dt.weekday
test['dayofyear'] = test['date'].dt.dayofyear
test['quarter'] = test['date'].dt.quarter
test['is_month_start'] = test['date'].dt.is_month_start
test['is_month_end']= test['date'].dt.is_month_end
test['is_quarter_start'] = test['date'].dt.is_quarter_start
test['is_quarter_end'] = test['date'].dt.is_quarter_end
test['is_year_start'] = test['date'].dt.is_year_start
test['is_year_end'] = test['date'].dt.is_year_end
test.dtypes

<ipython-input-9-eaae50ded542>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.

<ipython-input-9-eaae50ded542>:5: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



id                           int64
date                datetime64[ns]
store                        int64
item                         int64
year                         int64
month                        int64
day                          int64
week                         int64
weekofyear                   int64
dayofweek                    int64
weekday                      int64
dayofyear                    int64
quarter                      int64
is_month_start                bool
is_month_end                  bool
is_quarter_start              bool
is_quarter_end                bool
is_year_start                 bool
is_year_end                   bool
dtype: object

In [10]:
#Create mask for boolean values for test dataset
test['is_month_start'] = test['is_month_start'].replace({True: 1, False: 0})
test['is_month_end'] = test['is_month_end'].replace({True: 1, False: 0})
test['is_quarter_start'] = test['is_quarter_start'].replace({True: 1, False: 0})
test['is_quarter_end'] = test['is_quarter_end'].replace({True: 1, False: 0})
test['is_year_start'] = test['is_year_start'].replace({True: 1, False: 0})
test['is_year_end'] = test['is_year_end'].replace({True: 1, False: 0})
test['is_year_end'] = test['is_year_end'].astype(str).astype(np.int64)

#Add sales column with merge/join function to test data
def merge(x,y,col,col_name):
    x =pd.merge(x, y, how='left', on=None, left_on=col, right_on=col,
            left_index=False, right_index=False, sort=True,
             copy=True, indicator=False,validate=None)
    
    x=x.rename(columns={'sales':col_name})
    return x

test=merge(test, daily_avg,['item','store','dayofweek'],'daily_avg')
test=merge(test, monthly_avg,['item','store','month'],'monthly_avg')
test=merge(test, mean_store_item_month, ['month','item','store'],'mean_store_item_month')
test=merge(test, item_month_sum,['month','item'],'item_month_sum')
test=merge(test, store_month_sum,['month','store'],'store_month_sum')
#test=merge(test, store_item_shifted_365,['item','store'],'store_item_shifted_365')
#test=merge(test, item_week_shifted_90,['weekofyear', 'item'],'item-week_shifted_90')

#convert float to int
test['daily_avg']=test['daily_avg'].astype(np.int64)
test['monthly_avg']=test['monthly_avg'].astype(np.int64)
test['mean_store_item_month']=test['mean_store_item_month'].astype(np.int64)
#test['store_item_shifted_365']=test['store_item_shifted_365'].astype(np.int64)
test.dtypes

id                                int64
date                     datetime64[ns]
store                             int64
item                              int64
year                              int64
month                             int64
day                               int64
week                              int64
weekofyear                        int64
dayofweek                         int64
weekday                           int64
dayofyear                         int64
quarter                           int64
is_month_start                    int64
is_month_end                      int64
is_quarter_start                  int64
is_quarter_end                    int64
is_year_start                     int64
is_year_end                       int64
daily_avg                         int64
monthly_avg                       int64
mean_store_item_month             int64
item_month_sum                    int64
store_month_sum                   int64
dtype: object

In [11]:
test.head()

,id,date,store,item,year,month,day,week,weekofyear,dayofweek,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,daily_avg,monthly_avg,mean_store_item_month,item_month_sum,store_month_sum
0,0,2018-01-01,1,1,2018,1,1,1,1,0,...,0,1,0,1,0,15,13,13,22987,249352
1,7,2018-01-08,1,1,2018,1,8,2,2,0,...,0,0,0,0,0,15,13,13,22987,249352
2,14,2018-01-15,1,1,2018,1,15,3,3,0,...,0,0,0,0,0,15,13,13,22987,249352
3,21,2018-01-22,1,1,2018,1,22,4,4,0,...,0,0,0,0,0,15,13,13,22987,249352
4,28,2018-01-29,1,1,2018,1,29,5,5,0,...,0,0,0,0,0,15,13,13,22987,249352


In [12]:
# As we can see there are no missing values.
train_NA = train.isna()
train_num_NA = train_NA.sum()
train_num_NA

date                     0
store                    0
item                     0
sales                    0
year                     0
month                    0
day                      0
week                     0
weekofyear               0
dayofweek                0
weekday                  0
dayofyear                0
quarter                  0
is_month_start           0
is_month_end             0
is_quarter_start         0
is_quarter_end           0
is_year_start            0
is_year_end              0
daily_avg                0
monthly_avg              0
mean_store_item_month    0
item_month_sum           0
store_month_sum          0
dtype: int64

In [13]:
test= test.drop(['date','id'], axis = 1)
train= train.drop(['date'], axis = 1)

### Modeling

* #### LGBM

In [14]:
train_size = int(len(train) * 0.66)
train_n, test_n = train[0:train_size], train[train_size:len(train)]
print('Observations: %d' % (len(train)))
print('Training Observations: %d' % (len(train_n)))
print('Testing Observations: %d' % (len(test_n)))

train_n = pd.DataFrame(train_n)
test_n = pd.DataFrame(test_n)

x_train = train_n.drop('sales', axis=1)
y_train = train_n.pop('sales')
x_test = test_n.drop('sales', axis=1)
y_test = test_n.pop('sales')


Observations: 913000
Training Observations: 602580
Testing Observations: 310420


In [15]:
from sklearn.metrics import mean_squared_error
x_train,x_test,y_train,y_test = train_test_split(train.drop('sales',axis=1),train.pop('sales'),random_state=42,test_size=0.2)
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {'task':'train', 'boosting_type':'gbdt', 'objective':'regression', 
              'metric': {'rmse'}, 'num_leaves': 10, 'learning_rate': 0.05, 
              'feature_fraction': 0.8, 'max_depth': 5, 'verbose': 0, 
              'num_boost_round':15000, 'early_stopping_rounds':100, 'nthread':-1}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

# predict
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

C:\Users\chinchu\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

C:\Users\chinchu\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.102054 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's rmse: 27.5796
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 26.455
[3]	valid_0's rmse: 25.3853
[4]	valid_0's rmse: 24.3781
[5]	valid_0's rmse: 23.4218
[6]	valid_0's rmse: 22.5193
[7]	valid_0's rmse: 21.667
[8]	valid_0's rmse: 20.8541
[9]	valid_0's rmse: 20.0812
[10]	valid_0's rmse: 19.3613
[11]	valid_0's rmse: 18.6796
[12]	valid_0's rmse: 18.0289
[13]	valid_0's rmse: 17.4128
[14]	valid_0's rmse: 16.8383
[15]	valid_0's rmse: 16.2822
[16]	valid_0's rmse: 15.7634
[17]	valid_0's rmse: 15.2685
[18]	valid_0's rmse: 14.8092
[19]	valid_0's rmse: 14.3728
[20]	valid_0's rmse: 13.9651
[21]	valid_0's rmse: 13.5725
[22]	valid_0's rmse: 13.201
[23]	valid_0's rmse: 12.8592
[24]	valid_0's rmse: 12.5287
[25]	valid_0's rmse: 12.2

In [16]:
y_test = gbm.predict(test,num_iteration=gbm.best_iteration)
sample['sales'] = y_test
sample.to_csv('submission2.csv', index=False)

In [17]:
sample.head()

,id,sales
0,0,12.440209
1,1,12.315210
2,2,12.219655
3,3,12.223092
4,4,12.223637


#### End of file reached!!!!